In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
print(tf.__version__)

2.10.0


In [3]:
fashion_mnist = keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

In [5]:
# our next task is to scale our data
train_images = train_images/255.0
test_images = test_images/255.0

In [6]:
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [7]:
train_images[0].shape

(28, 28)

In [15]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [16]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [17]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [18]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [19]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 20m 44s]
val_accuracy: 0.8698333501815796

Best val_accuracy So Far: 0.909500002861023
Total elapsed time: 01h 18m 42s
INFO:tensorflow:Oracle triggered exit


In [20]:
model = tuner_search.get_best_models(num_models=1)[0]   

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 112)       1120      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 48)        134448    
                                                                 
 flatten (Flatten)           (None, 23232)             0         
                                                                 
 dense (Dense)               (None, 32)                743456    
                                                                 
 dense_1 (Dense)             (None, 10)                330       
                                                                 
Total params: 879,354
Trainable params: 879,354
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 411s 243ms/step - loss: 0.1647 - accuracy: 0.9399 - val_loss: 0.2687 - val_accuracy: 0.9055
Epoch 5/10
1688/1688 [==============================] - 370s 219ms/step - loss: 0.1300 - accuracy: 0.9521 - val_loss: 0.2664 - val_accuracy: 0.9090
Epoch 6/10
1688/1688 [==============================] - 394s 234ms/step - loss: 0.1039 - accuracy: 0.9617 - val_loss: 0.2863 - val_accuracy: 0.9127
Epoch 7/10
1688/1688 [==============================] - 404s 240ms/step - loss: 0.0805 - accuracy: 0.9701 - val_loss: 0.3361 - val_accuracy: 0.9127
Epoch 8/10
1688/1688 [==============================] - 361s 214ms/step - loss: 0.0627 - accuracy: 0.9773 - val_loss: 0.3441 - val_accuracy: 0.9102
Epoch 9/10
1688/1688 [==============================] - 354s 210ms/step - loss: 0.0530 - accuracy: 0.9801 - val_loss: 0.4144 - val_accuracy: 0.9058
Epoch 10/10
1688/1688 [==============================] - 356s 211ms/step - loss: 0.0406 - accuracy: 0.9856 - val